In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

/home/paul/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-17 12:09:05.311372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 12:09:06.931492: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = load_dataset("SetFit/SentEval-CR")

Repo card metadata block was not found. Setting CardData to empty.
Extracting data files: 100%|█| 2/2 [00:00<00:00, 392.08it/
Generating train split: 3012 examples [00:00, 80956.38 examples/s]
Generating test split: 753 examples [00:00, 162841.50 examples/s]


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3012
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 753
    })
})

In [15]:
dataset['train'][1000]

{'text': 'the customer support is pathetic .',
 'label': 0,
 'label_text': 'negative'}

In [11]:
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

In [13]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [16]:
trainer.train()
metrics = trainer.evaluate()

Generating Training Pairs: 100%|█| 20/20 [00:00<00:00, 119
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16
Epoch: 100%|████████████████| 1/1 [00:10<00:00, 10.67s/it]
***** Running evaluation *****


In [18]:
metrics

{'accuracy': 0.849933598937583}